<a href="https://colab.research.google.com/github/Jairex22/Google-AI-Burger-picture/blob/main/Te_damos_la_bienvenida_a_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os, re, ast, operator as op
from typing import List, Tuple
import numpy as np
import faiss

from transformers import pipeline
from sentence_transformers import SentenceTransformer, util

In [3]:
# Embeddings for retrieval
embed = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")   # 384-dim, fast

# QA over context
qa = pipeline("question-answering", model="deepset/roberta-base-squad2")

# Summarization
summ = pipeline("summarization", model="facebook/bart-large-cnn")

# Zero-shot classification for routing
zsc = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [4]:
# Embeddings for retrieval
embed = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")   # 384-dim, fast

# QA over context
qa = pipeline("question-answering", model="deepset/roberta-base-squad2")

# Summarization
summ = pipeline("summarization", model="facebook/bart-large-cnn")

# Zero-shot classification for routing
zsc = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

Device set to use cpu
Device set to use cpu
Device set to use cpu


In [5]:
corpus = [
  ("agentic_ai.txt", "Agentic AI combines LLMs with tools, memory, and goals to act autonomously."),
  ("rag.txt",        "Retrieval-Augmented Generation grounds answers in external documents to reduce hallucinations."),
  ("langchain.txt",  "LangChain orchestrates prompts, tools, memory, and retrieval for building LLM apps."),
  ("langgraph.txt",  "LangGraph models stateful agent workflows with explicit nodes and edges."),
  ("crewai.txt",     "CrewAI coordinates multi-agent collaboration with roles and delegated tasks.")
]

docs = [t for _, t in corpus]
metas = [n for n, _ in corpus]

# Build FAISS
embs = embed.encode(docs, convert_to_numpy=True, normalize_embeddings=True)
index = faiss.IndexFlatIP(embs.shape[1])     # cosine via normalized dot-product
index.add(embs)

def retrieve(query:str, k:int=3) -> List[Tuple[str,str,float]]:
    qv = embed.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    D, I = index.search(qv, k)
    out = []
    for d,i in zip(D[0], I[0]):
        out.append((metas[i], docs[i], float(d)))
    return out

In [6]:
OPS = {ast.Add: op.add, ast.Sub: op.sub, ast.Mult: op.mul, ast.Div: op.truediv,
       ast.Pow: op.pow, ast.USub: op.neg, ast.Mod: op.mod}

def safe_calc(expr:str) -> str:
    node = ast.parse(expr, mode="eval").body
    def eval_(x):
        if isinstance(x, ast.Num): return x.n
        if isinstance(x, ast.UnaryOp) and type(x.op) in OPS: return OPS[type(x.op)](eval_(x.operand))
        if isinstance(x, ast.BinOp)   and type(x.op) in OPS: return OPS[type(x.op)](eval_(x.left), eval_(x.right))
        raise ValueError("Disallowed expression")
    return str(eval_(node))

In [7]:
def tool_rag_qa(question:str, k:int=3) -> str:
    hits = retrieve(question, k=k)
    contexts = [t for _, t, _ in hits]
    best_ans, best_score, best_src = "", -1.0, None
    for ctx, (src, _, _) in zip(contexts, hits):
        out = qa(question=question, context=ctx)
        if out.get("score",0) > best_score:
            best_score, best_ans, best_src = out["score"], out["answer"], src
    cite = f" (source: {best_src})" if best_src else ""
    return (best_ans or "No answer found") + cite

def tool_summarize(text:str, max_words:int=120) -> str:
    raw = summ(text, max_length=min(256, max_words*2), min_length=max(40, max_words//2), do_sample=False)[0]["summary_text"]
    return raw

def tool_classify(text:str, labels:List[str]) -> str:
    res = zsc(text, candidate_labels=labels, multi_label=False)
    return f"Label: {res['labels'][0]} (score {res['scores'][0]:.2f})"

In [8]:
ROUTES = ["CALC", "RAG_QA", "SUMMARIZE", "CLASSIFY", "FREEFORM"]

def route(query:str) -> str:
    # quick heuristics
    if re.search(r"\d+\s*[\+\-\*\/]\s*\d+", query): return "CALC"
    if any(k in query.lower() for k in ["who", "what", "why", "how", "when", "compare", "explain"]): return "RAG_QA"
    if "summarize" in query.lower(): return "SUMMARIZE"
    if any(k in query.lower() for k in ["classify","category","label"]): return "CLASSIFY"
    # fallback to zero-shot
    zs = zsc(query, candidate_labels=ROUTES, multi_label=False)
    return zs["labels"][0]

In [9]:
def agent(query:str) -> str:
    r = route(query)
    if r == "CALC":
        m = re.search(r"([\d\.\s\+\-\*\/\(\)]+)", query)
        expr = m.group(1) if m else query
        try:
            return safe_calc(expr)
        except Exception as e:
            return f"Calc error: {e}"

    if r == "RAG_QA":
        return tool_rag_qa(query, k=3)

    if r == "SUMMARIZE":
        # try to summarize retrieved context if no explicit text provided
        m = re.search(r"summarize[:\- ](.+)", query, re.I)
        text = m.group(1) if m else " ".join(t for _, t, _ in retrieve(query, k=2))
        return tool_summarize(text, max_words=120)

    if r == "CLASSIFY":
        m = re.search(r"labels?:\s*(.+)$", query, re.I)
        labels = [l.strip() for l in (m.group(1).split(","))] if m else ["agents","rag","frameworks"]
        text = re.sub(r"labels?:.+","", query, flags=re.I)
        return tool_classify(text, labels)

    # FREEFORM fallback: answer from nearest doc with a short summary
    hits = retrieve(query, k=1)
    ctx = hits[0][1] if hits else ""
    return tool_summarize(ctx if ctx else "I don't have context.", max_words=80)


In [10]:
print("\nQ1: What is RAG and why is it useful for agents?")
print("A1:", agent("What is RAG and why is it useful for agents?"))

print("\nQ2: Compute 15*(4+2)")
print("A2:", agent("Compute 15*(4+2)"))

print("\nQ3: Summarize: Agentic AI combines LLMs with tools, memory, and goals to act autonomously.")
print("A3:", agent("Summarize: Agentic AI combines LLMs with tools, memory, and goals to act autonomously."))

print("\nQ4: Classify this text into labels agents, rag, frameworks: LangChain orchestrates prompts, tools, memory, and retrieval.")
print("A4:", agent("Classify this text into labels agents, rag, frameworks: LangChain orchestrates prompts, tools, memory, and retrieval. Labels: agents, rag, frameworks"))

print("\nQ5: Compare LangChain and LangGraph in one sentence.")
print("A5:", agent("Compare LangChain and LangGraph in one sentence."))


Q1: What is RAG and why is it useful for agents?


Your max_length is set to 240, but your input_length is only 21. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=10)


A1: roles (source: crewai.txt)

Q2: Compute 15*(4+2)
A2: Calc error: unexpected indent (<unknown>, line 1)

Q3: Summarize: Agentic AI combines LLMs with tools, memory, and goals to act autonomously.
A3: Agentic AI combines LLMs with tools, memory, and goals to act autonomously. Agentic AI is a new form of AI developed by the University of California, San Diego. It is the result of a collaboration between the university and Google's artificial intelligence lab, Cogent AI.

Q4: Classify this text into labels agents, rag, frameworks: LangChain orchestrates prompts, tools, memory, and retrieval.
A4: Label: agents (score 0.37)

Q5: Compare LangChain and LangGraph in one sentence.
A5: LangGraph models stateful agent workflows with explicit nodes and edges (source: langgraph.txt)
